In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# متباينة CHSH

*تقدير الاستخدام: دقيقتان على معالج Heron r2 (ملاحظة: هذا تقدير فقط. قد يختلف وقت التشغيل الفعلي لديك.)*
## الخلفية النظرية
في هذا البرنامج التعليمي، ستُجري تجربة على حاسوب كمومي لإثبات انتهاك متباينة CHSH باستخدام الأداة الأولية Estimator.

متباينة CHSH، المسماة نسبةً إلى مؤلفيها Clauser وHorne وShimony وHolt، تُستخدم لإثبات نظرية Bell تجريبياً (1969). تؤكد هذه النظرية أن نظريات المتغيرات الخفية المحلية لا يمكنها تفسير بعض نتائج التشابك الكمومي في ميكانيكا الكم. يُستخدم انتهاك متباينة CHSH لإثبات أن ميكانيكا الكم غير متوافقة مع نظريات المتغيرات الخفية المحلية. وتُعدّ هذه التجربة بالغة الأهمية لفهم أسس ميكانيكا الكم.

في عام 2022، مُنحت جائزة نوبل في الفيزياء إلى Alain Aspect وJohn Clauser وAnton Zeilinger، جزئياً تقديراً لأعمالهم الرائدة في علم المعلومات الكمومية، ولا سيما تجاربهم مع الفوتونات المتشابكة التي تُثبت انتهاك متباينات Bell.
## المتطلبات
قبل البدء بهذا البرنامج التعليمي، تأكد من تثبيت ما يلي:

* Qiskit SDK الإصدار 1.0 أو أحدث، مع دعم [التصور البياني](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) الإصدار 0.22 أو أحدث
## الإعداد

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## الخطوة 1: تحويل المدخلات الكلاسيكية إلى مسألة كمومية
في هذه التجربة، سننشئ زوجاً متشابكاً نقيس فيه كل كيوبت على أساسين مختلفين. سنسمي الأساسين للكيوبت الأول $A$ و$a$، والأساسين للكيوبت الثاني $B$ و$b$. يتيح لنا ذلك حساب كمية CHSH $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

كل راصد يأخذ قيمة $+1$ أو $-1$. من الواضح أن أحد الحدين $B\pm b$ يساوي $0$، والآخر يساوي $\pm 2$. لذلك، $S_1 = \pm 2$. يجب أن تُحقق القيمة المتوسطة لـ $S_1$ المتباينة:

$$
|\langle S_1 \rangle|\leq 2.
$$

بتوسيع $S_1$ بدلالة $A$ و$a$ و$B$ و$b$ نحصل على:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

يمكنك تعريف كمية CHSH أخرى $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

وهذا يؤدي إلى متباينة أخرى:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

إذا كان يمكن وصف ميكانيكا الكم بنظريات المتغيرات الخفية المحلية، فيجب أن تتحقق المتباينتان السابقتان. غير أنه كما يُوضح هذا البرنامج التعليمي، يمكن انتهاك هاتين المتباينتين في الحاسوب الكمومي. وبذلك، تكون ميكانيكا الكم غير متوافقة مع نظريات المتغيرات الخفية المحلية.
إذا أردت التعمق أكثر في النظرية، استكشف [التشابك في التطبيق](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) مع John Watrous.
ستنشئ زوجاً متشابكاً بين كيوبتين في الحاسوب الكمومي عن طريق إنشاء حالة Bell $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. باستخدام الأداة الأولية Estimator، يمكنك الحصول مباشرةً على قيم التوقع المطلوبة ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$, و$\langle ab \rangle$) لحساب قيم التوقع لكميتي CHSH $\langle S_1\rangle$ و$\langle S_2\rangle$. قبل إدخال الأداة الأولية Estimator، كان يتعين عليك استنتاج قيم التوقع من نتائج القياسات.

ستقيس الكيوبت الثاني في الأساسين $Z$ و$X$. أما الكيوبت الأول فسيُقاس أيضاً في أساسين متعامدين، لكن بزاوية نسبةً إلى الكيوبت الثاني، وسنجري مسحاً لهذه الزاوية بين $0$ و$2\pi$. كما ستلاحظ، تجعل الأداة الأولية Estimator تشغيل الدوائر الكمومية ذات المعاملات سهلاً للغاية. بدلاً من إنشاء سلسلة من دوائر CHSH، تحتاج فقط إلى إنشاء دائرة CHSH *واحدة* بمعامل يحدد زاوية القياس وسلسلة من قيم الطور لهذا المعامل.

أخيراً، ستحلل النتائج وترسمها مقابل زاوية القياس. ستلاحظ أنه في نطاقات معينة من زوايا القياس، تتجاوز قيم التوقع لكميتي CHSH الحد $|\langle S_1\rangle| > 2$ أو $|\langle S_2\rangle| > 2$، مما يُثبت انتهاك متباينة CHSH.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### إنشاء دائرة CHSH ذات معاملات
أولاً، نكتب الدائرة بالمعامل $\theta$ الذي نسميه `theta`. يمكن للأداة الأولية [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) أن تُبسّط بشكل كبير بناء الدوائر وتحليل المخرجات عن طريق توفير قيم التوقع للرواصد مباشرةً. كثير من المسائل ذات الاهتمام، لا سيما التطبيقات قصيرة المدى على الأنظمة ذات الضوضاء، يمكن صياغتها بدلالة قيم التوقع. يمكن للأداة الأولية `Estimator` (V2) تغيير أساس القياس تلقائياً بناءً على الراصد المقدَّم.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### إنشاء قائمة بقيم الطور لتعيينها لاحقاً
بعد إنشاء دائرة CHSH ذات المعاملات، ستنشئ قائمة بقيم الطور لتعيينها للدائرة في الخطوة التالية. يمكنك استخدام الكود التالي لإنشاء قائمة من 21 قيمة طور تتراوح من $0$ إلى $2 \pi$ بتباعد متساوٍ، أي $0$، $0.1 \pi$، $0.2 \pi$، ...، $1.9 \pi$، $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### الرواصد
نحتاج الآن إلى رواصد لحساب قيم التوقع منها. في حالتنا، ننظر في أساسين متعامدين لكل كيوبت، مع السماح لدوران $Y$ ذي المعاملات في الكيوبت الأول بمسح أساس القياس بشكل شبه مستمر نسبةً إلى أساس الكيوبت الثاني. لذلك سنختار الرواصد $ZZ$ و$ZX$ و$XZ$ و$XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## الخطوة 2: تحسين المسألة لتنفيذها على العتاد الكمومي

لتقليل إجمالي وقت تنفيذ المهام، تقبل الأدوات الأولية V2 فقط الدوائر والرواصد المتوافقة مع التعليمات والتوصيلية المدعومة من قبل النظام المستهدف (يُشار إليها بـ ISA circuits وobservables، أي دوائر ورواصد معمارية مجموعة التعليمات).

### دائرة ISA

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### رواصد ISA

وبالمثل، نحتاج إلى تحويل الرواصد لجعلها متوافقة مع الواجهة الخلفية قبل تشغيل المهام مع [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). يمكننا إجراء التحويل باستخدام التابع `apply_layout` الخاص بكائن `SparsePauliOp`.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## الخطوة 3: التنفيذ باستخدام الأدوات الأولية لـ Qiskit
لتنفيذ التجربة بأكملها في استدعاء واحد لـ [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
يمكننا إنشاء أداة أولية [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) لحساب قيم التوقع لدينا. تأخذ طريقة `EstimatorV2.run()` متسلسلةً من `primitive unified blocs (PUBs)`. كل PUB عبارة عن متسلسلة بالصيغة `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)